In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm
import plotly.graph_objects as go
import pandas as pd


<h3> Calcolo concentrazioni reali

In [2]:
V_i = [1500.1, 1502.8, 1510, 1502.1, 1497, 1504.4, 1511, 1554, 1513, 1512, 1543]
V_B = [1513.4, 1497.5, 1514, 1534.1, 1504, 1518, 1507.1, 1550, 1504, 1514.8, 1545.4]
V_f = [v_i + v_b for v_i, v_b in zip(V_i, V_B)]
C_i = [58.195]  # micromolare
C_f = [] 

def calculate_C_f(V_i, V_f, C_i):
    return C_i * V_i / V_f

# Calcola le concentrazioni finali 
for i in range(len(V_i)):
    if i == 0:
        C_f_value = calculate_C_f(V_i[i], V_f[i], C_i[0])
    else:
        C_f_value = calculate_C_f(V_i[i], V_f[i], C_f_value)
    C_f.append(C_f_value)


C_f_reale = C_f[2:]
print(C_f_reale)
#concentration = [7.5, 3.75, 1.125, 0.5, 0.25, 0.1, 0.05, 0.025, 0.00125]


[7.245466497324707, 3.5845514872641604, 1.7880951604246744, 0.8900246027471149, 0.44558734791785914, 0.22308077920887665, 0.11187312527114032, 0.05588481743424216, 0.027920694631859748]


In [5]:
# Lista delle colonne da leggere per ogni concentrazione
columns = [
    (0, 1), (2, 3), (4, 5), (6, 7), (8, 9), (10, 11), (12, 13), (14, 15), (16, 17), (18, 19)
]

wavelengths = []
intensities = []

# Ciclo per leggere i dati da ciascuna coppia di colonne
for col_pair in columns:
    data = pd.read_csv(
        'concentrazioni.csv',
        header=1,
        usecols=col_pair,
        names=['Lughezza donda', 'Intensità'],
        sep=',',
        decimal='.',
        skipinitialspace=True,
        nrows=291
    )
    wavelengths.append(data['Lughezza donda'])
    intensities.append(data['Intensità'])

# Assegna i dati alle variabili corrispondenti
Wavelength_75, Wavelength_375, Wavelength_1125, Wavelength_05, Wavelength_025, Wavelength_01, Wavelength_005, Wavelength_0025, Wavelength_000125, Wavelength_fondo = wavelengths
Intensity_75, Intensity_375, Intensity_1125, Intensity_05, Intensity_025, Intensity_01, Intensity_005, Intensity_0025, Intensity_000125, Intensity_fondo = intensities


# Elimina le ultime 2 colonne da wavelengths e intensities
wavelengths = wavelengths[:-3]
intensities = intensities[:-3]

# Aggiorna anche le variabili individuali se necessario
Wavelength_75, Wavelength_375, Wavelength_1125, Wavelength_05, Wavelength_025, Wavelength_01, Wavelength_005 = wavelengths
Intensity_75, Intensity_375, Intensity_1125, Intensity_05, Intensity_025, Intensity_01, Intensity_005 = intensities


In [6]:
# Sottrazione del fondo da ogni coppia di Wavelength e Intensity
corrected_intensities1 = []

for intensity, fondo in zip(intensities, Intensity_fondo):
    corrected_intensity1 = intensity - fondo
    corrected_intensities1.append(corrected_intensity1)

    # Crea un DataFrame Pandas con i valori corretti
    df_corrected = pd.DataFrame(corrected_intensities1).transpose()
    # Define labels if not already defined
    if 'labels' not in locals():
        labels = [f"{value:.3g}" for value in C_f_reale[:-1]]  # Adjust based on the context of C_f_reale



In [7]:
 # Stampa la tabella
df_corrected_no_fondo = df_corrected.iloc[:, :-1]  # Rimuove l'ultima colonna che rappresenta il fondo
display(df_corrected_no_fondo)

,Intensità,Intensità,Intensità,Intensità,Intensità,Intensità
0,3.500750,1.707255,2.343808,0.972800,2.160814,0.584602
1,3.497508,1.679802,2.724172,1.574643,2.345600,0.705614
2,3.608907,1.301299,2.112738,1.012408,2.218534,0.807022
3,3.489175,1.347922,2.294178,1.000743,1.700895,0.275808
4,3.264441,1.181034,2.167069,0.893306,1.376085,0.535782
...,...,...,...,...,...,...
286,9.478392,7.352423,4.840966,2.560408,1.242702,0.606053
287,9.006232,8.008359,4.457921,2.644195,1.205572,0.814538
288,8.845861,7.286301,4.794618,2.339236,0.717757,0.466159
289,8.598531,7.138248,4.537270,2.363506,1.180928,0.788421


In [8]:
fig = go.Figure()

for i in range(df_corrected_no_fondo.shape[1]):
    fig.add_trace(go.Scatter(
        x=wavelengths[0],
        y=df_corrected_no_fondo.iloc[:, i],
        mode='lines',
        name=labels[i]
    ))

fig.update_layout(
    xaxis_title='Lunghezza d\'onda (nm)',
    yaxis_title='Intensità (a.u.)',
    template='plotly_white'
)

fig.show()


<h3> Accoppiamento curve

In [9]:
# Accoppio le code e creo un dataframe con i dati aggiornati
minimi = np.array([df_corrected_no_fondo.iloc[:, col].min() for col in range(df_corrected_no_fondo.shape[1])])
min_minimo = min(minimi)
offset = minimi - min_minimo  # Calculate the offset for each column
print("Offset:", offset)
for idx, col in enumerate(df_corrected_no_fondo.columns):
    df_corrected_no_fondo[col] = df_corrected_no_fondo[col] - offset[idx]

# Ensure the number of columns in df_corrected_no_fondo matches the number of labels
if df_corrected_no_fondo.shape[1] != len(labels[:-1]):
    print(f"Adjusting the number of labels to match the columns in df_corrected_no_fondo.")
    labels = labels[:df_corrected_no_fondo.shape[1]]  # Adjust labels to match the number of columns

# Crea una tabella Pandas con i valori corretti
labels_matched = labels[:df_corrected_no_fondo.shape[1]]  # Ensure label count matches columns
df_corrected_table = pd.DataFrame(df_corrected_no_fondo.values, columns=labels_matched)
display(df_corrected_table)

fig = go.Figure()

# Aggiungi le tracce per ogni colonna del dataframe corretto
for col, label in zip(df_corrected_table.columns, labels[:-1]):  # Exclude the last label as it corresponds to the background
    fig.add_trace(go.Scatter(x=wavelengths[0], y=df_corrected_table[col], mode='lines', name=f'Concentrazione {label}'))

# Aggiungi titolo e etichette degli assi
fig.update_layout(
    title='Curve accoppiate',
    xaxis_title='Lunghezza d\'onda (nm)',
    yaxis_title='Intensità corretta (a.u.)',
    template='plotly_white'
)

# Mostra il grafico
fig.show()

Offset: [2.50884423 0.79409129 0.80401769 0.51491192 0.37709365 0.        ]
Adjusting the number of labels to match the columns in df_corrected_no_fondo.


,7.25,3.58,1.79,0.89,0.446,0.223
0,-1.498209,-3.291703,-2.655151,-4.026159,-2.838145,-4.414357
1,-1.501451,-3.319157,-2.274787,-3.424316,-2.653358,-4.293345
2,-1.390052,-3.697660,-2.886221,-3.986551,-2.780424,-4.191937
3,-1.509784,-3.651037,-2.704781,-3.998216,-3.298064,-4.723151
4,-1.734517,-3.817925,-2.831889,-4.105653,-3.622874,-4.463176
...,...,...,...,...,...,...
286,4.479433,2.353464,-0.157992,-2.438551,-3.756257,-4.392906
287,4.007274,3.009400,-0.541037,-2.354764,-3.793387,-4.184421
288,3.846902,2.287342,-0.204341,-2.659722,-4.281202,-4.532800
289,3.599572,2.139289,-0.461689,-2.635452,-3.818031,-4.210537


<h3> Fit parabolici

In [11]:

data_sets = [
    (Wavelength_75, Intensity_75), 
    (Wavelength_375, Intensity_375),  
    (Wavelength_1125, Intensity_1125),
    (Wavelength_05, Intensity_05),
    (Wavelength_025, Intensity_025),
    (Wavelength_01, Intensity_01)
]

In [12]:
# Palette pastello e vivid: stessi colori, vivid più saturi
pastel_colors = ['#a3c1da', '#f7cac9', '#b5ead7', '#c7ceea', '#ffdac1', '#e2f0cb', '#b2d8b2']
vivid_colors = ['#1976d2', '#e75480', '#00b894', '#7c43bd', '#ff9800', '#ffd600', '#388e3c']
max_fits = []

def fit_closest_to_max(x, y, num_points=20):
    # Fit a parabola to the num_points closest to the max value
    max_idx = np.argmax(y)
    max_val = y[max_idx]
    distances = np.abs(y - max_val)
    closest_indices = np.argsort(distances)[:num_points]
    x_fit = x.iloc[closest_indices] if hasattr(x, "iloc") else x[closest_indices]
    y_fit = y.iloc[closest_indices] if hasattr(y, "iloc") else y[closest_indices]
    # Fit a parabola: y = a*(x - x0)**2 + IMAX
    def parabola(x, x0, a, IMAX):
        return a * (x - x0) ** 2 + IMAX
    popt, _ = curve_fit(parabola, x_fit, y_fit, p0=[x_fit.iloc[np.argmax(y_fit)] if hasattr(x_fit, "iloc") else x_fit[np.argmax(y_fit)], -1, max_val])
    return popt  # returns (x0, a, IMAX)

def max_fit_parabolic(x, x0, a, IMAX):
    return a * (x - x0) ** 2 + IMAX

fig = go.Figure()

for i, (x_data, y_data) in enumerate(data_sets):
    color = pastel_colors[i % len(pastel_colors)]
    vivid_color = vivid_colors[i % len(vivid_colors)]
    params = fit_closest_to_max(x_data, y_data, num_points=20)
    λcenter_fit, a_fit, IMAX_fit = params

    max_fits.append(IMAX_fit)  # Salva il massimo della parabola

    # Dati originali: linea pastello
    fig.add_trace(go.Scatter(
        x=x_data, y=y_data, mode='lines', name=f'Data set {i+1}',
        line=dict(color=color, width=2, dash='dash')
    ))

    # Fit: linea vivida
    max_index = np.argmax(y_data)
    max_intensity = y_data[max_index]
    distances = np.abs(y_data - max_intensity)
    closest_indices = np.argsort(distances)[:20]
    x_closest = x_data[closest_indices]
    x_fit = np.linspace(min(x_closest), max(x_closest), 100)
    y_fit = max_fit_parabolic(x_fit, *params)
    fig.add_trace(go.Scatter(
        x=x_fit, y=y_fit, mode='lines', name=f'Fit {i+1}',
        line=dict(color=vivid_color, width=2)
    ))

fig.update_layout(template='plotly_white')
fig.show()


In [13]:
I0 = max_fits[-1]
Cans = 4.969

def funzione_binding(P, Δη, n, KD):
    par = n*P - Cans + KD
    sqrt = np.sqrt(par**2 - (4 * P * n * Cans))
    return (Δη/2) * (par - sqrt) + I0

In [21]:
# Rimuovere l'ultimo valore da C_f_reale e max_fits per allineare le lunghezze
P = C_f_reale[:-3]
y = max_fits

# Esegui il fit con curve_fit, aggiungendo bounds per aiutare la convergenza
initial_params = [max(y), 1, np.median(P)]  # Δη = max(y), n = 1, KD = valore mediano di P
bounds = ([0, 0, 0], [np.inf, 10, np.inf])  # Δη > 0, 0 < n < 10, KD > 0
popt, pcov = curve_fit(funzione_binding, P, y, p0=initial_params, bounds=bounds, maxfev=20000)

# Parametri ottimizzati
Δη_fit, n_fit, KD_fit = popt

# Crea un array di valori per P per il grafico del fit
P_fit = np.linspace(min(P), max(P), 100)
y_fit = funzione_binding(P_fit, Δη_fit, n_fit, KD_fit)

# Crea il grafico con Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=P, y=y, mode='markers', name='Dati originali'))
fig.add_trace(go.Scatter(x=P_fit, y=y_fit, mode='lines', name='Curva di fit'))
fig.update_layout(
    title='Fit dei dati con funzione binding',
    xaxis_title='Concentration',
    yaxis_title='IMAX',
    template='plotly_white'
)
fig.show()

# Crea un DataFrame Pandas con i risultati del fit
fit_results_df = pd.DataFrame({
    'Parameter': ['Δη', 'n', 'KD'],
    'Value': [Δη_fit, n_fit, KD_fit]
})
print(fit_results_df)

/tmp/ipykernel_18590/675179864.py:6: RuntimeWarning:

invalid value encountered in sqrt



ValueError: Residuals are not finite in the initial point.